<a href="https://colab.research.google.com/github/david4096/quantum-examples/blob/main/Grovers_Algorithm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pennylane

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 20.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 930.0/930.0 kB 30.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 27.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 27.1 MB/s eta 0:00:00


In [11]:
import pennylane as qml
import numpy as np

# Create a device with 3 qubits
dev = qml.device('default.qubit', wires=3)

# Oracle to mark state |101⟩
def oracle():
    # Mark state |101⟩ with a negative phase
    # Use a simpler construction with direct phase flips
    qml.PauliX(wires=1)  # NOT on middle qubit (since we want |0⟩)
    qml.Toffoli(wires=[0, 1, 2])  # CCX to mark the state
    qml.PauliX(wires=1)  # Reset middle qubit

def diffusion():
    # Implement the standard diffusion operator
    # H⊗n → (2|0⟩⟨0| - I) → H⊗n
    for wire in range(3):
        qml.Hadamard(wires=wire)

    for wire in range(3):
        qml.PauliX(wires=wire)

    # Apply phase flip on |111⟩
    qml.Toffoli(wires=[0, 1, 2])

    for wire in range(3):
        qml.PauliX(wires=wire)

    for wire in range(3):
        qml.Hadamard(wires=wire)

@qml.qnode(dev)
def grover_circuit(iterations):
    # Initialize in equal superposition
    for wire in range(3):
        qml.Hadamard(wires=wire)

    # Apply Grover iterations
    for _ in range(iterations):
        oracle()
        diffusion()

    # Return probabilities directly
    return qml.probs(wires=range(3))

# Calculate optimal number of iterations
N = 2**3
M = 1  # searching for one marked state
iterations = 2  # For 3 qubits, 2 iterations is optimal

# Run the circuit
probabilities = grover_circuit(iterations)

# Print results
print(f"\nResults after {iterations} iterations:")
for i, prob in enumerate(probabilities):
    state = format(i, '03b')
    print(f"|{state}⟩: {prob:.3f}")

print(f"\nTarget state |101⟩ probability: {probabilities[5]:.3f}")  # 5 is binary 101


Results after 2 iterations:
|000⟩: 0.125
|001⟩: 0.000
|010⟩: 0.125
|011⟩: 0.000
|100⟩: 0.125
|101⟩: 0.500
|110⟩: 0.125
|111⟩: 0.000

Target state |101⟩ probability: 0.500


In [16]:
import pennylane as qml
import numpy as np

# Create a device with an arbitrary number of qubits
def create_device(num_qubits):
    return qml.device('default.qubit', wires=num_qubits)

# Generalized Oracle to mark an arbitrary target state
def oracle(target_state):
    # Apply X gates to qubits where the target state has a 0
    for i, bit in enumerate(target_state):
        if bit == '0':
            qml.PauliX(wires=i)

    # Apply multi-controlled Toffoli (CCX for n qubits)
    control_wires = list(range(len(target_state) - 1))
    target_wire = len(target_state) - 1
    qml.MultiControlledX(wires=control_wires + [target_wire], control_values=[1] * (len(target_state) - 1))

    # Reset X gates to return qubits to original state
    for i, bit in enumerate(target_state):
        if bit == '0':
            qml.PauliX(wires=i)

# Generalized Diffusion operator for n qubits
def diffusion(num_qubits):
    # H⊗n → (2|0⟩⟨0| - I) → H⊗n
    for wire in range(num_qubits):
        qml.Hadamard(wires=wire)

    for wire in range(num_qubits):
        qml.PauliX(wires=wire)

    # Apply phase flip on |111...111⟩ (n qubits all 1)
    control_wires = list(range(num_qubits - 1))
    qml.MultiControlledX(wires=control_wires + [num_qubits - 1], control_values=[1] * (num_qubits - 1))

    for wire in range(num_qubits):
        qml.PauliX(wires=wire)

    for wire in range(num_qubits):
        qml.Hadamard(wires=wire)

# Grover's algorithm circuit for arbitrary target state and number of iterations
def grover_circuit(target_state, iterations, num_qubits):
    dev = create_device(num_qubits)

    @qml.qnode(dev)
    def circuit():
        # Initialize in equal superposition
        for wire in range(num_qubits):
            qml.Hadamard(wires=wire)

        # Apply Grover iterations
        for _ in range(iterations):
            oracle(target_state)
            diffusion(num_qubits)

        # Return probabilities directly
        return qml.probs(wires=range(num_qubits))

    return circuit()

# Function to calculate optimal number of iterations for Grover's algorithm
def optimal_iterations(num_qubits, num_marked_states=1):
    N = 2**num_qubits
    return int(np.floor(np.pi / 4 * np.sqrt(N / num_marked_states)))

# Set parameters
num_qubits = 18  # For example, let's use 5 qubits
target_state = '101011101011101011'  # The target state we want to mark
iterations = optimal_iterations(num_qubits)

# Run the Grover circuit with the specified number of qubits, target state, and iterations
probabilities = grover_circuit(target_state, iterations, num_qubits)

# Print results
print(f"\nResults after {iterations} iterations:")
for i, prob in enumerate(probabilities):
    state = format(i, f'0{num_qubits}b')
    print(f"|{state}⟩: {prob:.3f}")

# Print the probability of the target state
target_index = int(target_state, 2)
print(f"\nTarget state |{target_state}⟩ probability: {probabilities[target_index]:.3f}")


Streaming output truncated to the last 5000 lines.
|111110110001111010⟩: 0.000
|111110110001111011⟩: 0.000
|111110110001111100⟩: 0.000
|111110110001111101⟩: 0.000
|111110110001111110⟩: 0.000
|111110110001111111⟩: 0.000
|111110110010000000⟩: 0.000
|111110110010000001⟩: 0.000
|111110110010000010⟩: 0.000
|111110110010000011⟩: 0.000
|111110110010000100⟩: 0.000
|111110110010000101⟩: 0.000
|111110110010000110⟩: 0.000
|111110110010000111⟩: 0.000
|111110110010001000⟩: 0.000
|111110110010001001⟩: 0.000
|111110110010001010⟩: 0.000
|111110110010001011⟩: 0.000
|111110110010001100⟩: 0.000
|111110110010001101⟩: 0.000
|111110110010001110⟩: 0.000
|111110110010001111⟩: 0.000
|111110110010010000⟩: 0.000
|111110110010010001⟩: 0.000
|111110110010010010⟩: 0.000
|111110110010010011⟩: 0.000
|111110110010010100⟩: 0.000
|111110110010010101⟩: 0.000
|111110110010010110⟩: 0.000
|111110110010010111⟩: 0.000
|111110110010011000⟩: 0.000
|111110110010011001⟩: 0.000
|111110110010011010⟩: 0.000
|111110110010011011⟩: 0.0